In [2]:
import pandas as pd
import json
import os

import warnings 
warnings.filterwarnings("ignore")
from importlib import reload

# this ensures that I can update the class without losing my variables in my notebook
import xenium_cluster
reload(xenium_cluster)
from xenium_cluster import XeniumCluster
from utils.metrics import *

In [3]:
# Path to your .gz file
file_path = 'data/hBreast/transcripts.csv.gz'

# Read the gzipped CSV file into a DataFrame
df_transcripts = pd.read_csv(file_path, compression='gzip')

# drop cells without ids
df_transcripts = df_transcripts[df_transcripts["cell_id"] != -1]

# drop blanks and controls
df_transcripts = df_transcripts[~df_transcripts["feature_name"].str.startswith('BLANK_') & ~df_transcripts["feature_name"].str.startswith('NegControl')]

In [8]:
def run_experiment(data, dataset_name: str, current_spot_size: int, third_dim: bool, resolutions: list):
    
    clustering = XeniumCluster(data=data, dataset_name=dataset_name)
    clustering.set_spot_size(current_spot_size)
    clustering.create_spot_data(third_dim=third_dim, save_data=True)

    print(f"The size of the spot data is {clustering.xenium_spot_data.shape}")

    clustering.normalize_counts(clustering.xenium_spot_data)
    clustering.generate_neighborhood_graph(clustering.xenium_spot_data, plot_pcas=False)

    Leiden_cluster = clustering.Leiden(clustering.xenium_spot_data, resolutions, embedding="umap", save_plot=True)

    return clustering, Leiden_cluster

In [9]:
import numpy as np
import torch
from scipy.spatial.distance import cdist

def record_results(original_data, cluster_dict, results_dir, model_name, filename, spot_size, third_dim, K=None, resolution=None, uses_spatial=True):

    if resolution is not None:
        current_clustering = np.array(cluster_dict[model_name][spot_size][third_dim].get(
            resolution, 
            cluster_dict[model_name][spot_size][third_dim]
        ))
    else:
        current_clustering = np.array(cluster_dict[model_name][spot_size][third_dim][uses_spatial].get(
            K, 
            cluster_dict[model_name][spot_size][third_dim][uses_spatial]
        ))
    cluster_labels = np.unique(current_clustering)

    original_data.xenium_spot_data.obs[f"{model_name} cluster"] = np.array(current_clustering)
    dirpath = f"{results_dir}/{model_name}{'/' + (str(resolution) if resolution is not None else str(K))}/clusters/{spot_size}"
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

    filepath = f"{dirpath}/{filename}{resolution}.csv"

    original_data.xenium_spot_data.obs[f"{model_name} cluster"].to_csv(filepath)
    # Extracting row, col, and cluster values from the dataframe
    rows = torch.tensor(original_data.xenium_spot_data.obs["row"].astype(int))
    cols = torch.tensor(original_data.xenium_spot_data.obs["col"].astype(int))
    clusters = torch.tensor(original_data.xenium_spot_data.obs[f"{model_name} cluster"].astype(int))
    cluster_labels = np.unique(clusters)

    num_rows = int(max(rows) - min(rows) + 1)
    num_cols = int(max(cols) - min(cols) + 1)

    cluster_grid = torch.zeros((num_rows, num_cols), dtype=torch.int)

    cluster_grid[rows, cols] = torch.tensor(clusters, dtype=torch.int)

    wss = {}
    for label in cluster_labels:
        current_cluster_locations = torch.stack(torch.where((cluster_grid == label)), axis=1).to(float)
        wss[f"Cluster {label}"] = (spot_size ** 2) * torch.mean(torch.cdist(current_cluster_locations, current_cluster_locations)).item()
        print(f"POSSIBLE {len(cluster_labels)}", label, wss[f"Cluster {label}"])

    wss_dirpath = f"{results_dir}/{model_name}{'/' + (str(resolution) if resolution is not None else str(K))}/wss/{spot_size}/"
    if not os.path.exists(wss_dirpath):
        os.makedirs(wss_dirpath)

    wss_filepath = f"{wss_dirpath}/{filename}_wss.json"
    with open(wss_filepath, "w") as f:
        json.dump(wss, f, indent=4)

In [10]:
cluster_dict = {"Leiden": {}}
wss = {"Leiden": {}}
results_dir = "results/hBreast"

In [11]:
import matplotlib
matplotlib.use('Agg')

resolutions = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0]
for spot_size in [50, 75, 100]:
    for third_dim in [False]:
        cluster_results_filename = f"clusters_RES="
        original_data, Leiden_cluster = run_experiment(df_transcripts, "hBreast", spot_size, third_dim, resolutions)

        # Leiden
        if "Leiden" not in cluster_dict:
            cluster_dict["Leiden"] = {}
        if spot_size not in cluster_dict["Leiden"]:
            cluster_dict["Leiden"][spot_size] = {}
        cluster_dict["Leiden"][spot_size][third_dim] = {res: clusters.tolist() for res, clusters in Leiden_cluster.items()}
        for resolution in resolutions:
            record_results(original_data, cluster_dict, results_dir, "Leiden", cluster_results_filename, spot_size, third_dim, resolution=resolution)

        print(f"Cluster with spot size {(spot_size, third_dim)} completed.")

The size of the spot data is (23444, 280)
POSSIBLE 2 0 237268.92648201477
POSSIBLE 2 1 208733.4746680792
POSSIBLE 6 0 243131.44239847283
POSSIBLE 6 1 191996.0711914089
POSSIBLE 6 2 189237.71286891235
POSSIBLE 6 3 201496.97160171438
POSSIBLE 6 4 76551.93663019121
POSSIBLE 6 5 52211.06193422333
POSSIBLE 13 0 269844.45895805577
POSSIBLE 13 1 179640.51826966458
POSSIBLE 13 2 175890.100149156
POSSIBLE 13 3 145678.45302802633
POSSIBLE 13 4 192216.5390880566
POSSIBLE 13 5 126403.4883344763
POSSIBLE 13 6 211221.2004144602
POSSIBLE 13 7 80230.3326617599
POSSIBLE 13 8 188230.96094653106
POSSIBLE 13 9 188905.3311258151
POSSIBLE 13 10 58646.964944213854
POSSIBLE 13 11 198613.74582987558
POSSIBLE 13 12 159435.33617198112
POSSIBLE 17 0 277673.82549326535
POSSIBLE 17 1 131092.93569597206
POSSIBLE 17 2 178479.31557889085
POSSIBLE 17 3 211635.95056765532
POSSIBLE 17 4 187061.97214653384
POSSIBLE 17 5 173569.203612595
POSSIBLE 17 6 141951.81672229522
POSSIBLE 17 7 143227.41391836116
POSSIBLE 17 8 187675

In [12]:
spot_sizes = [50,75,100]
resolutions = [0.25, 0.5, 0.75, 1.0]
in_billions = 1_000_000_000
method = "Leiden"
for spot_size in spot_sizes:
    for K in [17]:
        for resolution in resolutions:
            filename = f"results/hBreast/{method}/{resolution}/wss/{spot_size}/{cluster_results_filename}_wss.json"
            if os.path.exists(filename):
                with open(filename, "r") as wss_dict:
                    current_wss = json.load(wss_dict)
                print("Method:", method, "Spot Size", spot_size, "Resolution", resolution, "Num Clusters:", len(current_wss), "Total WSS", sum(current_wss.values()) / in_billions)

Method: Leiden Spot Size 50 Resolution 0.25 Num Clusters: 6 Total WSS 0.0009546251966249229
Method: Leiden Spot Size 50 Resolution 0.5 Num Clusters: 13 Total WSS 0.0021749574299220723
Method: Leiden Spot Size 50 Resolution 0.75 Num Clusters: 17 Total WSS 0.0027965800966310134
Method: Leiden Spot Size 50 Resolution 1.0 Num Clusters: 20 Total WSS 0.003262291797661095
Method: Leiden Spot Size 75 Resolution 0.25 Num Clusters: 5 Total WSS 0.0014419638080543848
Method: Leiden Spot Size 75 Resolution 0.5 Num Clusters: 11 Total WSS 0.0026451834152004194
Method: Leiden Spot Size 75 Resolution 0.75 Num Clusters: 14 Total WSS 0.003391287264966858
Method: Leiden Spot Size 75 Resolution 1.0 Num Clusters: 17 Total WSS 0.004020492087786737
Method: Leiden Spot Size 100 Resolution 0.25 Num Clusters: 6 Total WSS 0.0023271593834089
Method: Leiden Spot Size 100 Resolution 0.5 Num Clusters: 9 Total WSS 0.002881843232376044
Method: Leiden Spot Size 100 Resolution 0.75 Num Clusters: 14 Total WSS 0.0045784615